In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# Set paths for training and validation data
train_dir = 'C:/Users/User/chest_xray/train'
validation_dir = 'C:/Users/User/chest_xray/test'


In [3]:
# Create ImageDataGenerators for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [16]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Set up early stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [17]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
    #,
    #callbacks=[early_stopping]
)

# Save the model
model.save('chest_xray_classifier.h5')


Epoch 1/10
163/163 [==============================] - 103s 624ms/step - loss: 0.4221 - accuracy: 0.8308 - val_loss: 0.4355 - val_accuracy: 0.8043
Epoch 2/10
163/163 [==============================] - 101s 620ms/step - loss: 0.2652 - accuracy: 0.8896 - val_loss: 0.5597 - val_accuracy: 0.7549
Epoch 3/10
163/163 [==============================] - 102s 622ms/step - loss: 0.2067 - accuracy: 0.9167 - val_loss: 0.2806 - val_accuracy: 0.8865
Epoch 4/10
163/163 [==============================] - 101s 619ms/step - loss: 0.1967 - accuracy: 0.9225 - val_loss: 0.3330 - val_accuracy: 0.8832
Epoch 5/10
163/163 [==============================] - 100s 610ms/step - loss: 0.1817 - accuracy: 0.9288 - val_loss: 0.5408 - val_accuracy: 0.7993
Epoch 6/10
163/163 [==============================] - 100s 611ms/step - loss: 0.1693 - accuracy: 0.9381 - val_loss: 0.4223 - val_accuracy: 0.8618
Epoch 7/10
163/163 [==============================] - 108s 663ms/step - loss: 0.1570 - accuracy: 0.9362 - val_loss: 0.3083 -

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('chest_xray_classifier.h5')

def preprocess_image(img_path):
    """Preprocess the image for prediction."""
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale pixel values to [0, 1]
    return img_array

def predict_image(img_path):
    """Predict if the image is Normal or PNEUMONIA."""
    img_array = preprocess_image('C:/Users/User/my_img/43.jpeg')
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        return "PNEUMONIA"
    else:
        return "Normal"

# Test the model with a new image
new_image_path = 'path_to_new_image.jpg'
result = predict_image(new_image_path)
print(f'The predicted class for the image is: {result}')


1/1 [==============================] - 0s 55ms/step
The predicted class for the image is: Normal


In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel, QFileDialog, QVBoxLayout
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('chest_xray_classifier.h5')

def preprocess_image(img_path):
    """Preprocess the image for prediction."""
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale pixel values to [0, 1]
    return img_array

def predict_image(img_path):
    """Predict if the image is Normal or PNEUMONIA."""
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        return "PNEUMONIA"
    else:
        return "Normal"

class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'Chest X-Ray Classifier'
        self.left = 100
        self.top = 100
        self.width = 800
        self.height = 600
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        layout = QVBoxLayout()

        self.label = QLabel(self)
        self.label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.label)
        
        self.result_label = QLabel('Prediction: ', self)
        self.result_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.result_label)
        
        self.button = QPushButton('Open Image', self)
        self.button.clicked.connect(self.open_image)
        layout.addWidget(self.button)
        
        self.setLayout(layout)
        
        self.show()

    def open_image(self):
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        file_name, _ = QFileDialog.getOpenFileName(self, 'Open Image', '', 'Image Files (*.png *.jpg *.jpeg)', options=options)
        if file_name:
            self.display_image(file_name)
            prediction = predict_image(file_name)
            self.result_label.setText(f'Prediction: {prediction}')

    def display_image(self, file_path):
        pixmap = QPixmap(file_path)
        self.label.setPixmap(pixmap.scaled(self.label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())





1/1 [==============================] - 0s 16ms/step


SystemExit: 0

C:\Users\User\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
